# Customizing nbconvert

Under the hood, nbconvert uses [Jinja templates](http://jinja.pocoo.org/docs/latest/) to specify how the notebooks should be formatted. These templates can be fully customized, allowing you to use nbconvert to create notebooks in different formats with different styles as well.

## Converting a notebook to an (I)Python script and printing to stdout

Out of the box, nbconvert can be used to convert notebooks to plain Python files. For example, the following command converts the `example.ipynb` notebook to Python and prints out the result:

In [ ]:
!jupyter nbconvert --to python 'example.ipynb' --stdout

From the code, you can see that non-code cells are also exported. If you wanted to change that behaviour, you would first look to nbconvert [configuration options page](./config_options.rst) to see if there is an option available that can give you your desired behaviour. 

In this case, if you wanted to remove non-code cells from the output, you could use the `TemplateExporter.exclude_markdown` traitlet directly, as below. 

In [ ]:
!jupyter nbconvert --to python 'example.ipynb' --stdout --TemplateExporter.exclude_markdown=True

## Custom Templates 

As mentioned above, if you want to change this behavior, you can use a custom template.  The custom template inherits from the Python template and overwrites the markdown blocks so that they are empty. 

Below is an example of a custom template, which we write to a file called `simplepython.py.j2`. This template removes markdown cells from the output, and also changes how the execution count numbers are formatted:

In [ ]:
%%writefile simplepython.py.j2

{% extends 'python/index.py.j2'%}

## remove markdown cells
{% block markdowncell %}
{% endblock markdowncell %}

## change the appearance of execution count
{% block in_prompt %}
# [{{ cell.execution_count if cell.execution_count else ' ' }}]:
{% endblock in_prompt %}

Using this template, we see that the resulting Python code does not contain anything that was previously in a markdown cell, and only displays execution counts (i.e., `[#]:` not `In[#]:`):

In [ ]:
!jupyter nbconvert --to python 'example.ipynb' --stdout --template-file=simplepython.py.j2

### Saving Custom Templates

By default, nbconvert finds templates from a few locations.

The recommended place to save custom templates, so that they are globally accessible to nbconvert, is your jupyter data directories:

- share/jupyter
    - nbconvert
        - templates
            - base
            - classic
            - lab
            - latex
            - markdown

The HTML and LaTeX/PDF exporters will search the lab and latex subdirectories for templates by default, respectively.

To find your jupyter configuration directory you can use:

```python
from jupyter_core.paths import jupyter_path
print(jupyter_path('nbconvert','templates'))
```

Additionally,

```python
TemplateExporter.template_path=['.']
```

defines an additional list of paths that nbconvert can look for user defined templates. It defaults to searching for custom templates in the current working directory and can be changed through configuration options.

## Template structure

Nbconvert templates consist of a set of nested blocks. When defining a new
template, you extend an existing template by overriding some of the blocks.

All the templates shipped in nbconvert have the basic structure described here,
though some may define additional blocks.

In [ ]:
from IPython.display import HTML
with open('template_structure.html') as f:
    display(HTML(f.read()))

See also [template_structure.html](template_structure.html).

### A few gotchas

Jinja uses `%`, `{`, and `}` for syntax by default which does not play nicely with LaTeX. In LaTeX, we have the following replacements:

| Syntax   | Default | LaTeX   |
|----------|---------|---------|
| block    | {% %}   | ((* *)) |
| variable | {{ }}   | ((( ))) |
| comment  | {# #}   | ((= =)) |

## Templates using cell tags

The notebook file format supports attaching arbitrary JSON metadata to each cell. In addition, every cell has a special `tags` metadata field that accepts a list of strings that indicate the cell's tags. To apply these, go to the `View → CellToolbar → Tags` option which will create a Tag editor at the top of every cell. 

First choose a notebook you want to convert to html, and apply the tags: `"Easy"`, `"Medium"`, or 
`"Hard"`.  

With this in place, the notebook can be converted using a custom template.

Design your template in the cells provided below.

Hint: tags are located at `cell.metadata.tags`, the following Python code collects the value of the tag: 

```python
cell['metadata'].get('tags', [])
```

Which you can then use inside a Jinja template as in the following:

In [ ]:
%%writefile mytemplate.html.j2

{% extends 'lab/index.html.j2'%}
{% block any_cell %}
{% if 'Hard' in cell['metadata'].get('tags', []) %}
    <div style="border:thin solid red">
        {{ super() }}
    </div>
{% elif 'Medium' in cell['metadata'].get('tags', []) %}
    <div style="border:thin solid orange">
        {{ super() }}
    </div>
{% elif 'Easy' in cell['metadata'].get('tags', []) %}
    <div style="border:thin solid green">
        {{ super() }}
    </div>
{% else %}
    {{ super() }}
{% endif %}
{% endblock any_cell %}

Now, if we collect the result of using nbconvert with this template, and display the resulting html, we see the following:

In [ ]:
example = !jupyter nbconvert --to html 'example.ipynb' --template-file=mytemplate.html.j2 --stdout
example = example[3:] # have to remove the first three lines which are not proper html
from IPython.display import HTML
display(HTML('\n'.join(example))) 

## Templates using custom cell metadata 

We demonstrated [above](#Templates-using-cell-tags) how to use cell tags in a template to apply custom styling to a notebook. But remember, the notebook file format supports attaching _arbitrary_ JSON metadata to each cell, not only cell tags. 
Here, we describe an exercise for using an `example.difficulty` metadata field (rather than cell tags) to do the same as before (to mark up different cells as being "Easy", "Medium" or "Hard").

### How to edit cell metadata

To edit the cell metadata from within the notebook, go to the menu item: `View → Cell Toolbar → Edit Metadata`. This will bring up a toolbar above each cell with a button that says "Edit Metadata". Click this button, and a field will pop up in which you will directly edit the cell metadata JSON. 

**NB**: Because it is JSON, you will need to ensure that what you write is valid JSON. 

### Template challenges: dealing with missing custom metadata fields

One of the challenges of dealing with custom metadata is to handle the case where the metadata is not present on every cell. This can get somewhat tricky because of JSON objects tendency to be deeply nested coupled with Python's (and therefore Jinja's) approach to calling into dictionaries. Specifically, the following code will error:

```python
foo = {}
foo["bar"]
```

Accordingly, it is better to use the [{}.get()](https://docs.python.org/3.6/library/stdtypes.html#dict.get) method which allows you to set a default value to return if no key is found as the second argument. 

Hint: if your metadata items are located at `cell.metadata.example.difficulty`, the following Python code would get the value defaulting to an empty string (`''`) if nothing is found:

```python
cell['metadata'].get('example', {}).get('difficulty', '')
```

### Exercise: Write a template for handling custom metadata
Now, write a template that will look for `Easy`, `Medium` and `Hard` metadata values for the `cell.metadata.example.difficulty` field and wrap them in a div with a green, orange, or red thin solid border (respectively). 

**NB**: This is the same design and logic as used in the previous cell tag example.

#### How to get `example.ipynb`

We have provided an example file in `example.ipynb` in the nbconvert documentation that has already been marked up with both tags and the above metadata for you to test with. You can get it from [this link to the raw file](example.ipynb) or by cloning the repository [from GitHub](https://github.com/jupyter/nbconvert) and navingating to `nbconvert/docs/source/example.ipynb`. 

#### Convert `example.ipynb` using cell tags 

First, make sure that you can reproduce the previous result using the cell tags template that we have provided above. 

**Easy**: If you want to make it easy on yourself, create a new file `my_template.html.j2` in the same directory as `example.ipynb` and copy the contents of the cell we use to write `mytemplate.html.j2` to the file system. 

Then run `jupyter nbconvert --to html 'example.ipynb' --template-file=mytemplate.html.j2` and see if your template file works. 


**Moderate**: If you want more of a challenge, try recreating the jinja template by modifying the following jinja template file:

```python
{% extends 'lab/base.html.j2'%}
{% block any_cell %}
    <div style="border:thin solid red">
        {{ super() }}
    </div>
{% endblock any_cell %}
```

**Moderate**: If you want custom config or a composition of files, you'll want to instead make a template directory. Create the directory `mytemplate` and add the following files to it:

- `index.html.j2` -- copy the contents of the cell we use to write `mytemplate.html.j2` to the file system.
- `conf.json` -- add the following contents to set the default configuration options
```json
{
    "base_template": "lab",
    "mimetypes": {
        "text/html": true
    }
}
```

Then run `jupyter nbconvert --to html 'example.ipynb' --template=mytemplate` and see if your package works. This option allows for much more flexibility and combination of configuration and additional config files to support more advanced templates.


**Hard**: If you want even more of a challenge, try recreating the jinja template from scratch. 

#### Write your template

Once you've done at least the **Easy** version of the previous step, try modifying your template to use `cell.metadata.example.difficulty` fields rather than cell tags. 

#### Convert `example.ipynb` with formatting from custom metadata

Once you've written your template, try converting `example.ipynb` using the following command (making sure that `your_template.html.j2` is in your local directory where you are running the command):

```bash
jupyter nbconvert --to html 'example.ipynb' --template=your_template.html.j2 --stdout
```

The resulting display should pick out different cells to be bordered with green, orange, or red.

If you do that successfullly, the resulting html document should look like the following
(see also [example-custom-metadata.html](example-custom-metadata.html)): 

In [ ]:
from IPython.display import IFrame

IFrame('example-custom-metadata.html', width='100%', height=600)